In [1]:
import sys
sys.path.append("../bus_service_increase")

In [2]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import altair as alt
import calitp_data_analysis.magics
import pandas as pd

from IPython.display import display, HTML

from bus_service_utils import chart_utils
from calitp_data_analysis import calitp_color_palette as cp
from update_vars import PUBLIC_FILENAME, YEAR, MONTH
#from monthly_ridership_by_rtpa import get_percent_change
from shared_utils.rt_dates import MONTH_DICT

#Temp file path for testing
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/csuyat_folder/"

#alt.renderers.enable("html")
alt.data_transformers.enable('default', max_rows=None)

In [3]:
# parameters cell for local
rtpa = "Metropolitan Transportation Commission"

In [4]:
%%capture_parameters
rtpa

{"rtpa": "Metropolitan Transportation Commission"}


# {rtpa}
## Monthly Ridership Trends

**Download data from our public [folder](https://console.cloud.google.com/storage/browser/calitp-publish-data-analysis)** by navigating to `ntd_monthly_ridership` and selecting a file.

In [5]:
URL = ("https://console.cloud.google.com/storage/"
       "browser/calitp-publish-data-analysis"
      )

display(
    HTML(
        f"""
        <a href={URL}>
        Download the latest month of data: {PUBLIC_FILENAME}</a>
        """
    )
)


In [6]:
#updated to filter for period_year greater than 2018
df = pd.read_parquet(
    f"{GCS_FILE_PATH}ca_monthly_ridership_{YEAR}_{MONTH}.parquet",
    filters = [[("RTPA", "==", rtpa), ("period_year", ">=", 2018)]]
).drop(
    #updated columns names to match new df
    columns = ["mode", "tos"]
).rename(columns = {"Mode_full": "Mode", "TOS_full": "TOS"})

In [7]:
# delete this
display(
    df.shape,
    df.columns,
)

(6004, 27)

Index(['uza_name', 'uace_cd', '_dt', 'ts', 'ntd_id', 'year', 'reporter_type',
       'agency', 'Status', 'mode_full_name', 'service_type', '_3_mode',
       'legacy_ntd_id', 'period_year_month', 'period_year', 'period_month',
       'upt', 'vrm', 'vrh', 'voms', 'RTPA', '_merge', 'previous_y_m_upt',
       'change_1yr', 'pct_change_1yr', 'Mode', 'TOS'],
      dtype='object')

In [8]:
# find columns that are recent enough to plot
MIN_YEAR = 2018

#might not need the rest of these
not_id_cols = [c for c in df.columns if "/" in c]

recent_years = [
    c for c in not_id_cols if int(c.split("/")[1]) >= MIN_YEAR and 
    "pct" not in c
]

upt_cols = [
    c for c in recent_years if "change" not in c
]

change_cols = [c for c in recent_years if "change" in c]

In [9]:
# do i need this anymore? 
# what does the data look like initially? filters/groups the DF by the `group_cols` list, keeps the initial wide data

# give this a try with current data NOPE! 
def OLD_sum_by_group(df: pd.DataFrame, group_cols: list) -> pd.DataFrame:
    """
    Since df is wide, use pivot_table() to sum up all
    the columns that show UPT.
    """
    grouped_df = df.pivot_table(
        index = group_cols, 
        values = "upt", 
        aggfunc="sum"
    ).reset_index().reindex(columns = group_cols + recent_years)
    
    return grouped_df


# DONT THINK I NEED THIS ANYMORE!!!
def make_long(df: pd.DataFrame, group_cols: list, value_cols: list):
    df_long = df[group_cols + value_cols].melt(
        id_vars = group_cols, 
        value_vars = value_cols,
    )
    
    df_long = df_long.assign(
        variable = df_long.variable.str.replace("change_1yr_", "")
    )
    
    return df_long

In [31]:
# UPDATED AND TESTED, this works! gives the same results as the old old sum_by_group, make_long and assemble_long_df functions

def sum_by_group(
    df: pd.DataFrame,
    group_cols: list) -> pd.DataFrame:
    """
    since data is now long to begin with, this replaces old sum_by_group, make_long and assemble_long_df functions.
    """
    from TEST_monthly_ridership_by_rtpa import get_percent_change
    grouped_df = df.groupby(group_cols+
                             ['period_year',
                             'period_month',
                             'period_year_month']
                           ).agg({
        "upt":"sum",
        "previous_y_m_upt":"sum",
        "change_1yr":"sum"
    }
    ).reset_index()
    
    #get %change back
    grouped_df = get_percent_change(grouped_df)
    
    return grouped_df
    

In [11]:
# takes the sum_by_group dfs and makes them long
# unpivots the sum_by_group dfs, each row is a different year/month

# DONT THINK I NEED THIS ANYMORE!!!
def assemble_long_df(df: pd.DataFrame, group_cols: list) -> pd.DataFrame:
    """
    Need df to be long to make chart.
    Let's put raw UPT and change side-by-side.
    """
    df_raw = make_long(df, group_cols, upt_cols).rename(
        columns = {"value": "upt"})
    df_change = make_long(df, group_cols, change_cols).rename(
        columns = {"value": "change_1yr"})

    final = pd.merge(
        df_raw,
        df_change,
        on = group_cols + ["variable"],
        how = "left"
    )
    
    final = final.assign(
        year = final.variable.str.split("/", expand=True)[1],
        month = final.variable.str.split("/", expand=True)[0].str.zfill(2)
    )
    
    final = final.assign(
        year_month = final.year + "-" + final.month
    )
    
    return final

In [12]:
# KEEPING AS IS FOR NOW, BUT MAY NEED TO REWORK 

def remove_zero_upt_rows(df: pd.DataFrame) -> pd.DataFrame:
    """
    takes the by_agency_long df, filters for rows with zero UPT AND zero change_1yr. 
    then removes these rows from the by_agency_long df.
    resulting df should only contain rows with change in UPT not due to (0 UPT - 0 UPT) and net zero change in UPT
    zero UPT and zero change_1yr occurs when (0 upt - 0upt = 0change), dont need to show this
    but a net zero change (100upt - 100upt = 0 change), should be shown
    """
    
    #df of rows with zero UPT for 2 years
    zero_UPT_2years = df[(df["upt"] == 0) & (df["change_1yr"] == 0)]
    
    merge = df.merge(zero_UPT_2years, how="left", indicator=True)
    no_zero_UPT_rows = merge[merge["_merge"] == "left_only"].drop(columns=["_merge"]).dropna(subset="change_1yr")
    
    return no_zero_UPT_rows
    

In [32]:
# NEW 

agency_cols = ["ntd_id", "agency", "RTPA"]
mode_cols = ["Mode", "RTPA"]
tos_cols = ["TOS", "RTPA"]

by_agency_long = sum_by_group(df, agency_cols)
by_mode_long = sum_by_group(df, mode_cols)
by_tos_long = sum_by_group(df, tos_cols)

# zero upt still works!
by_agency_long_no_zero_upt = remove_zero_upt_rows(by_agency_long)

In [33]:
# 
# this matches the initial script
by_agency_long[(by_agency_long["ntd_id"] == "90014") & (by_agency_long["period_year_month"] =="2024-01")]

,ntd_id,agency,RTPA,period_year,period_month,period_year_month,upt,previous_y_m_upt,change_1yr,pct_change_1yr
300,90014,Alameda-Contra Costa Transit District,Metropolitan Transportation Commission,2024,1,2024-01,3244890,2786363.0,458527.0,0.1413


In [15]:
by_agency_long_no_zero_upt["period_year"].value_counts().sort_values()

2024     80
2018    230
2019    240
2020    240
2021    240
2022    240
2023    240
Name: period_year, dtype: int64

In [16]:
#I STILL NEED FILTERED df BY AGENCY, MODE AND TOS

#by_agency_long = assemble_long_df(by_agency, agency_cols)
#by_mode_long = assemble_long_df(by_mode, mode_cols)
#by_tos_long = assemble_long_df(by_tos, tos_cols)

#by_agency_long_no_zero_upt = remove_zero_upt_rows(by_agency_long)

In [17]:
LABELING_DICT = {
    "upt": "Unlinked Passenger Trips",
    "change_1yr": "Change in Unlinked Passenger Trips from Prior Year",
    "TOS": "Type of Service",
    "year_month": "Date"
}

def labeling(word: str) -> str:
    return chart_utils.labeling(word, LABELING_DICT)

WIDTH = 300
HEIGHT = 150

In [18]:
def make_line_chart(
    df: pd.DataFrame, 
    y_col: str,
    color_col: str,
) -> alt.Chart:
    df = df[df[y_col] > 0].dropna(subset = y_col)

    x_label = [i for i in df.period_year_month.unique() if 
               any(substring in i for substring in 
                   ["-01", "-06"])
              ]    
    chart = (alt.Chart(df)
         .mark_line()
         .encode(
             x = alt.X("period_year_month:O", 
                       axis=alt.Axis(values = x_label), 
                       title = "Date"
                      ),
             y = alt.Y(y_col, title = labeling(y_col)),
             color = alt.Color(color_col, title = "", 
                               scale = alt.Scale(
                                   range = cp.CALITP_CATEGORY_BRIGHT_COLORS + 
                                   cp.CALITP_CATEGORY_BOLD_COLORS
                               )),
             tooltip = ["period_year_month", y_col, color_col, "RTPA"]
         ).properties(width = WIDTH, height = HEIGHT)
         .facet(color_col, columns=2, title = "")
         .resolve_scale(y="independent")
    ).properties(
        title = f"{labeling(y_col)} by {labeling(color_col)}"
    ).interactive()
        
    return chart

In [36]:
def make_bar_chart(
    df: pd.DataFrame, 
    y_col: str,
    color_col: str,
) -> alt.Chart:
    
    def short_label(word):
        shorten_dict = {
            "change_1yr": "Change",
            "pct_change_1yr":"Change",
        }
        return shorten_dict[word]
    
    # For change column, we are missing everything prior to 2023 
    #df = df.dropna(subset = y_col)
    
    #need flag for y_col >,<, 0, missing?
    #count function to how many agencies fall in those categories, then look at those agencies
    #present table
    
    x_label = [i for i in df.period_year_month.unique() if 
               any(substring in i for substring in 
                   ["-01", "-03", "-06", "-09"])
              ]
    
    chart = (alt.Chart(df)
         .mark_bar()
         .encode(
             x = alt.X("period_year_month:O", 
                       axis=alt.Axis(values = x_label), 
                       title = "Date"
                      ),
             y = alt.Y(y_col, title = short_label(y_col)),
             color = alt.Color(color_col, title = "", 
                              scale = alt.Scale(
                                   range = cp.CALITP_CATEGORY_BRIGHT_COLORS + 
                                   cp.CALITP_CATEGORY_BOLD_COLORS
                              )),
             tooltip = ["period_year_month", y_col, color_col, "RTPA"]
         ).properties(width = WIDTH, height = HEIGHT)
         .facet(color_col, columns=2, title = "")
         .resolve_scale(x="shared", 
                        y="independent")
    ).properties(
        title = f"{labeling(y_col)} by {labeling(color_col)}"
    ).interactive()
    
    return chart

### Transit Agency

In [20]:
make_line_chart(by_agency_long, y_col = "upt", color_col = "agency")

alt.FacetChart(...)

Change in Unlinked Passenger Trips from the prior year. For example, July 2023's change would be the change in July 2023's reported values against July 2022's reported values.

In [37]:
make_bar_chart(by_agency_long_no_zero_upt[by_agency_long_no_zero_upt["period_year"] >= 2024], y_col = "pct_change_1yr", color_col = "agency")

alt.FacetChart(...)

In [27]:
by_agency_long_no_zero_upt.columns

Index(['ntd_id', 'agency', 'RTPA', 'period_year', 'period_month',
       'period_year_month', 'upt', 'change_1yr'],
      dtype='object')

In [26]:
make_bar_chart(by_agency_long_no_zero_upt[by_agency_long_no_zero_upt["period_year"] >= 2024], y_col = "change_1yr", color_col = "agency")

alt.FacetChart(...)

### Transit Mode

In [22]:
make_line_chart(by_mode_long, y_col = "upt", color_col = "Mode")

alt.FacetChart(...)

In [23]:
make_bar_chart(by_mode_long[by_mode_long["period_year"] >= 2024], y_col = "change_1yr", color_col = "Mode")

alt.FacetChart(...)

### Type of Service

In [24]:
make_line_chart(by_tos_long, y_col = "upt", color_col = "TOS")

alt.FacetChart(...)

In [25]:
make_bar_chart(by_tos_long[by_tos_long["period_year"] >= 2024], y_col = "change_1yr", color_col = "TOS")

alt.FacetChart(...)